# What's this?

This is all the code that was necessary to transform the accepted loan offers, rejected loan offers, and transactional data into the final data format.

On a high level, we need to iterate through each row in the merchant data, and for each business in that row, we check the date of the application.
Then, we try to find the preceding 90 days' worth of transactions for that business, and calculate the total volume of transactions processed, and total successful and failed transactions.
We then print this "collated" form of the data to a new row in a new CSV file.

This shouldn't be necessary to do twice.

TODO: State the file name of the completed output file.

## Accepted transactions

In [24]:
import pandas
from datetime import datetime, timedelta

accepted = pandas.read_csv('preliminary_data/accepted_merchants.csv', index_col='buuid')
accepted_txn = pandas.read_csv('preliminary_data/accepted_txn.csv', index_col='buuid')

output_list = []

for business in accepted.itertuples():
    # The date that the plan was created is at index 8
    app_date = business[8]
    buuid = business[0]
    
    # Filter the accepted transactions for this business, 
    # and only consider the ones that are within 90 days before
    # the application date
    
    app_date_datetime = datetime.strptime(app_date.split(" ")[0], '%Y-%m-%d')
    app_date_minus_ninety = app_date_datetime - timedelta(days=90)

    app_date_str = app_date_datetime.date().strftime('%Y-%m-%d')
    app_date_minus_ninety_str = app_date_minus_ninety.date().strftime('%Y-%m-%d')
    
    bus_txns = accepted_txn.loc[[buuid]]
    filtered_txns = bus_txns[(bus_txns['transaction_date'] < app_date_str) & (bus_txns['transaction_date'] > app_date_minus_ninety_str)]
    
    # Now we have filtered the transactions we need to calculate the total transaction volume
    # As well as total amount of accepted and declined transactions

    business_total_volume = filtered_txns['transaction_value'].sum()
    business_total_accepted_txns = filtered_txns['approved_transaction_count'].sum()
    business_total_failed_txns = filtered_txns['failed_transaction_count'].sum()
    
    output_list.append([buuid, business_total_volume, business_total_accepted_txns, business_total_failed_txns, 1])

print('Writing the following table to CSV.')
output_df = pandas.DataFrame(output_list, columns=["buuid", "txn_vol", "accepted_txn", "failed_txn", "capital_granted"])
print(output_df.head(10))
output_df.to_csv('preliminary_data/accepted_output.csv', index=False)

Writing the following table to CSV.
                                               buuid     txn_vol  \
0  1467294299878-dfa9b9e8-5bec-4b43-bf20-2b75c95c...    96686.51   
1  1474379163995-fc8bbbc7-cf3f-4479-9335-3bc8adcc...  1405768.50   
2  1474379163995-fc8bbbc7-cf3f-4479-9335-3bc8adcc...  2285902.36   
3  1477904235403-44d3ebff-a66a-4136-9dd5-6556f70b...   288562.96   
4  1480063167618-82065c6c-5600-489f-9b03-db1cc86c...   234177.00   
5  1478782941341-f3898626-7f0c-4423-b236-75556a5c...   147860.00   
6  1483942481356-d108d2b0-e853-46e1-af8e-7f76034a...   688614.81   
7  1488481869839-b8e147d0-6541-4709-82b3-119f6d68...    35073.92   
8  1489089487227-1da43088-050a-4e76-aec7-ca82e045...   126152.00   
9  1490855005017-ea214d64-f92d-42cd-b5df-e27a2536...    71670.00   

   accepted_txn  failed_txn  capital_granted  
0            80          25                1  
1          1200         472                1  
2          1572         530                1  
3           323          68

## Rejected transactions

In [26]:
import pandas
from datetime import datetime, timedelta

# See the above accepted block for more thorough code comments.

failed = pandas.read_csv('preliminary_data/rejected_merchants.csv', index_col='buuid')
failed_txn = pandas.read_csv('preliminary_data/rejected_txn.csv', index_col='buuid')

output_list = []

for business in failed.itertuples():
    app_date = business[3]
    buuid = business.Index
    
    app_date_datetime = datetime.strptime(app_date.split(" ")[0], '%Y-%m-%d')
    app_date_minus_ninety = app_date_datetime - timedelta(days=90)

    app_date_str = app_date_datetime.date().strftime('%Y-%m-%d')
    app_date_minus_ninety_str = app_date_minus_ninety.date().strftime('%Y-%m-%d')
    
    try:
        bus_txns = failed_txn.loc[[buuid]]    
        filtered_txns = bus_txns[(bus_txns['transaction_date'] < app_date_str) & (bus_txns['transaction_date'] > app_date_minus_ninety_str)]
    
        business_total_volume = filtered_txns['transaction_value'].sum()
        business_total_accepted_txns = filtered_txns['approved_transaction_count'].sum()
        business_total_failed_txns = filtered_txns['failed_transaction_count'].sum()
    except KeyError:
        business_total_volume = 0
        business_total_accepted_txns = 0
        business_total_failed_txns = 0
        
    output_list.append([buuid, business_total_volume, business_total_accepted_txns, business_total_failed_txns, 0])

print('Writing the following table to CSV.')
output_df = pandas.DataFrame(output_list, columns=["buuid", "txn_vol", "accepted_txn", "failed_txn", "capital_granted"])
print(output_df.head(10))
output_df.to_csv('preliminary_data/rejected_output.csv', index=False)

Writing the following table to CSV.
                                               buuid     txn_vol  \
0  1484834667492-04ce1b02-9fc8-4e87-b164-dae5f5a0...  2455888.00   
1  1484834667492-04ce1b02-9fc8-4e87-b164-dae5f5a0...  2502993.00   
2  1484834667492-04ce1b02-9fc8-4e87-b164-dae5f5a0...  2453798.00   
3  1484834667492-04ce1b02-9fc8-4e87-b164-dae5f5a0...  2455888.00   
4  1484834667492-04ce1b02-9fc8-4e87-b164-dae5f5a0...  2455888.00   
5  1509109286074-0c36ba7d-023c-4694-85ec-3e0b6fdc...   762790.45   
6  1509109286074-0c36ba7d-023c-4694-85ec-3e0b6fdc...   762577.95   
7  1509109286074-0c36ba7d-023c-4694-85ec-3e0b6fdc...   762577.95   
8  1509109286074-0c36ba7d-023c-4694-85ec-3e0b6fdc...   762577.95   
9  1509109286074-0c36ba7d-023c-4694-85ec-3e0b6fdc...   762577.95   

   accepted_txn  failed_txn  capital_granted  
0          3420         385                0  
1          3440         370                0  
2          3435         380                0  
3          3420         385

## Zip up the two output files, and randomize their order

In [28]:
import pandas

accepted = pandas.read_csv('preliminary_data/rejected_output.csv', index_col='buuid')

rejected = pandas.read_csv('preliminary_data/accepted_output.csv', index_col='buuid')

collated = pandas.concat([accepted, rejected])

collated.sample(frac=1).to_csv('output_data/collated_output.csv', index=False)

print("Verily, your output is ready, m'lord.")

Verily, your output is ready, m'lord.
